# Toolhouse with Ollama

Now let's set up the Jupyter environment to handle asynchronous programming and enables the display of rich HTML content.

1. **`import nest_asyncio`**: Imports the `nest_asyncio` library, which allows for the smooth handling of asynchronous tasks in environments like Google Colab, where nested event loops might otherwise cause issues.

2. **`nest_asyncio.apply()`**: Applies a patch that enables running asynchronous code without conflicts. This is particularly useful when working with `asyncio` in notebooks, allowing for proper execution of asynchronous tasks.

3. **`from IPython.display import display, HTML`**: Imports functions from IPython’s display module that allow for the rendering of rich media content (like HTML) directly within the notebook. This enables the output of formatted HTML, which can be used for interactive content or visually enriched outputs.

This setup ensures that you can effectively run and display asynchronous code alongside HTML content in the same notebook.

In [1]:
import nest_asyncio

nest_asyncio.apply()

from IPython.display import display, HTML

Ensure you have Ollama installed

In [2]:
!which ollama

/usr/local/bin/ollama


To install Ollama: https://ollama.com/download

Install the ollama package from the Python Package Index (PyPI)

In [3]:
%pip install ollama --quiet

Note: you may need to restart the kernel to use updated packages.


Ensure you Ollama is running and Mistral is available in Ollama

In [5]:
!ollama list

NAME          	ID          	SIZE  	MODIFIED     
mistral:latest	f974a74358d6	4.1 GB	21 hours ago	
llama3:latest 	365c0bd3c000	4.7 GB	2 months ago	


To install Mistral on Ollama run the following cell

In [6]:
!ollama pull mistral

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling ff82381e2bea... 100% ▕████████████████▏ 4.1 GB                         
pulling 43070e2d4e53... 100% ▕████████████████▏  11 KB                         
pulling 491dfa501e59... 100% ▕████████████████▏  801 B                         
pulling ed11eda7790d... 100% ▕████████████████▏   30 B                         
pulling 42347cd80dc8... 100% ▕████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
success 


In [7]:
from getpass import getpass

# Prompt the user to enter the Exa API key securely
EXA_API_KEY = getpass("Enter your Exa API key: ")

Test the Exa AI search function using a direct web search query.

In [8]:
import json
import ollama
import asyncio
import requests

# Web search using Exa AI
def search_web(query: str) -> str:
  headers = {
    'accept': 'application/json',
    'content-type': 'application/json',
    'x-api-key': EXA_API_KEY,
  }
  data = {
    "query": query,
    "type": "neural",
    "useAutoprompt": True,
    "numResults": 10,
    "contents": {
      "text": True
    }
  }
  # Correct Exa AI search endpoint
  response = requests.post('https://api.exa.ai/search', headers=headers, json=data)
  
  # Check if the response is valid JSON
  try:
    response.raise_for_status()  # Raise an error for bad status codes
    response_json = response.json()
  except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
    return json.dumps({'error': 'HTTP error occurred'})
  except json.JSONDecodeError:
    print("Failed to decode JSON. Response content:")
    print(response.text)
    return json.dumps({'error': 'Failed to decode JSON response'})

  return json.dumps(response_json)

async def run(model: str):
  client = ollama.AsyncClient()
  # Initialize conversation with a user query
  messages = [{'role': 'user', 'content': 'Search the web for "current weather in New York"'}]

  # First API call: Send the query and function description to the model
  response = await client.chat(
    model=model,
    messages=messages,
    tools=[
      {
        'type': 'function',
        'function': {
          'name': 'search_web',
          'description': 'Search the web for a given query',
          'parameters': {
            'type': 'object',
            'properties': {
              'query': {
                'type': 'string',
                'description': 'The search query',
              },
            },
            'required': ['query'],
          },
        },
      },
    ],
  )

  # Add the model's response to the conversation history
  messages.append(response['message'])

  # Check if the model decided to use the provided function
  if not response['message'].get('tool_calls'):
    print("The model didn't use the function. Its response was:")
    print(response['message']['content'])
    return

  # Process function calls made by the model
  if response['message'].get('tool_calls'):
    available_functions = {
      'search_web': search_web,
    }
    for tool in response['message']['tool_calls']:
      function_to_call = available_functions[tool['function']['name']]
      function_response = function_to_call(tool['function']['arguments']['query'])
      # Add function response to the conversation
      messages.append(
        {
          'role': 'tool',
          'content': function_response,
        }
      )

  # Second API call: Get final response from the model
  final_response = await client.chat(model=model, messages=messages)
  print(final_response['message']['content'])


# Run the async function
asyncio.run(run('mistral'))

1. The weather forecast for the next two days is as follows:
   - Day 19 (Current day): Sunrise at 6:45 am, sunset at 5:35 pm. The UV index is 3 out of 10 and the skies are partly cloudy with a low temperature near 30F and winds WSW at 5 to 10 mph. The humidity level is 50%.
   - Day 20 (Next day): Sunrise at 6:43 am, sunset at 5:36 pm. The UV index is 3 out of 10 and the skies are a mix of clouds and sunshine with a high temperature of 46F and winds SW at 5 to 10 mph. The humidity level is 42%.

  2. Here are some articles related to your search:
   - Title: Articles about foggy | Gothamist
     URL: https://gothamist.com/tags/foggy
     Published Date: January 12, 2014
     Author: Ben Yakas
     Text Snippet: This article features beautiful photos of New York City enveloped in fog and includes a subscription form for essential NYC news, arts, events, and food delivered to your inbox daily.

   - Title: Articles about hot | Gothamist
     URL: https://gothamist.com/tags/hot
     Publ

Install the Toolhouse package from the Python Package Index (PyPI)

In [9]:
%pip install toolhouse --quiet

Note: you may need to restart the kernel to use updated packages.


In [11]:
from getpass import getpass

# Prompt the user to enter the Exa API key securely
TH_API_KEY = getpass("Enter your ToolHouse API key: ")

Check available tools in your personal ToolHouse "Tool Store" https://app.toolhouse.ai/

In [14]:
from toolhouse import Toolhouse

nest_asyncio.apply()

# Initialize Toolhouse
th = Toolhouse(access_token=TH_API_KEY)

# List available tools
available_tools = th.get_tools()
print("Available tools:")
for tool in available_tools:
    print(f"- {tool['function']['name']}")

Available tools:
- exa_web_search
- scraper


Test the toolhouse call with GPT-4o

In [17]:
!pip install toolhouse openai --quiet

import os
from openai import OpenAI
from toolhouse import Toolhouse
from getpass import getpass

# Set up API keys (you'll be prompted to enter these securely)
openai_api_key = getpass("Enter your OpenAI API key: ")
toolhouse_api_key = getpass("Enter your Toolhouse API key: ")

# Initialize OpenAI and Toolhouse clients
client = OpenAI(api_key=openai_api_key)
th = Toolhouse(access_token=toolhouse_api_key)

# Define the OpenAI model we want to use
MODEL = 'gpt-4'

# Set up the conversation
messages = [{
    "role": "user",
    "content": "Generate a Python function to calculate the factorial of a number, then use it to compute the factorial of 5."
}]

# First API call: Send the query and function description to the model
response = client.chat.completions.create(
    model=MODEL,
    messages=messages,
    tools=th.get_tools()  # This includes the Code Interpreter tool
)

# Add the model's response to the conversation history
messages.append(response.choices[0].message)

# Check if the model decided to use the provided tool
if response.choices[0].message.tool_calls:
    # Process function calls made by the model
    tool_responses = th.run_tools(response)
    messages.extend(tool_responses)

    # Second API call: Get final response from the model
    final_response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=th.get_tools()
    )
    
    print("Final response:")
    print(final_response.choices[0].message.content)
else:
    print("The model didn't use the tool. Its response was:")
    print(response.choices[0].message.content)

Test the toolhouse call with Ollama

In [16]:
import nest_asyncio
import json
import ollama
import asyncio

nest_asyncio.apply()

# temp fix
# th.set_base_url("https://t2ojtd5voi.execute-api.us-west-2.amazonaws.com/v1")

# Define the Ollama model we want to use
MODEL = 'mistral'

async def run_with_toolhouse(model: str):
    client = ollama.AsyncClient()
    # Initialize conversation with a user query
    messages = [{
        "role": "user",
        "content": "Search the web for current weather in New York using exa_web_search"
    }]

    # First API call: Send the query and function description to the model
    response = await client.chat(
        model=model,
        messages=messages,
        tools=th.get_tools(),  # This includes the Code Execution tool
    )

    print("Original response from Ollama:")
    print(json.dumps(response, indent=2))

    # Add the model's response to the conversation history
    messages.append(response['message'])

    # Check if the model decided to use the provided function
    if response['message'].get('tool_calls'):
        print("\nTool calls found in the response:")
        print(json.dumps(response['message']['tool_calls'], indent=2))

        # Adapt the response to match what Toolhouse expects
        adapted_response = {
            'choices': [{
                'message': response['message'],
                'finish_reason': 'tool_calls'
            }]
        }
        print("\nAdapted response for Toolhouse:")
        print(json.dumps(adapted_response, indent=2))

        # Process function calls made by the model
        try:
            tool_responses = th.run_tools(adapted_response)
            print("\nTool responses:")
            print(json.dumps(tool_responses, indent=2))
            messages.extend(tool_responses)
        except Exception as e:
            print(f"\nError running tools: {str(e)}")
            return

    # Second API call: Get final response from the model
    final_response = await client.chat(model=model, messages=messages)
    print("\nFinal response:")
    print(final_response['message']['content'])

# Run the async function
await run_with_toolhouse(MODEL)

UnprocessableEntityException: UnprocessableEntityException(status_code=422, message='Invalid data')